In [2]:
# 라이브러리 불러오기
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import json

import re

In [3]:
submission_data = pd.read_json('./data/test.json').transpose()
PRETRAINED_MODEL = "gogamza/kobart-base-v2"

In [4]:
# 전처리
def preprocess_sentence(sentence): 
    # \n을 공백으로 바꾸기
    sentence = re.sub("\n", " ", sentence)
    
    # (ㄱ-ㅎ, ㅏ-ㅣ, ".", "?", "!", ",", ' ')를 제외한 모든 문자를 없애기
    sentence = re.sub("[^ㄱ-ㅣ가-힣.?!, ]", "", sentence)
    
    # 단어와 구두점(punctuation) 사이에 공백 추가
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    
    return sentence


preprocessed = submission_data['text'].apply(preprocess_sentence).tolist()

In [5]:
# BART 분류기 클래스 정의하기
class BartForSequenceClassification(nn.Module):
    def __init__(self, num_labels):
        super(BartForSequenceClassification, self).__init__()
        self.num_labels = num_labels
        self.config = BartConfig.from_pretrained(PRETRAINED_MODEL, num_labels=num_labels)
        self.bart = BartModel.from_pretrained(PRETRAINED_MODEL, config=self.config)
        self.classifier = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(self.config.hidden_size, num_labels))
        
    def forward(self, *args, token_type_ids=None, labels=None, **kwargs, ): #input_ids, attention_mask=None, labels=None):
        outputs = self.bart(*args, **kwargs,)
        pooled_output = outputs[0][:, -1, :]  # 마지막 토큰의 출력 사용
        logits = self.classifier(pooled_output)
        
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        
        return loss, logits

NameError: name 'nn' is not defined

In [ ]:
import torch
from torch import nn
from transformers import BartModel, BartConfig



# 대화 최대 길이 128으로 설정
MAX_LENGTH = 128

from transformers import BartTokenizerFast, PreTrainedTokenizerFast
# 각 conversation을 토큰화하여 새로운 열 'tokenized'에 저장
tokenizer=PreTrainedTokenizerFast.from_pretrained(PRETRAINED_MODEL)

tokenized = tokenizer(
    preprocessed, 
    max_length=MAX_LENGTH,
    padding='max_length',  # Pad to the max_length
    truncation=True,       # Truncate sequences to the max_length
    return_tensors='pt')

In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('aiffel_minions/DLthon_finetune_koBart/model-epoch-best:v3', type='model')
artifact_dir = artifact.download()

In [ ]:
import torch
import os
model_path = os.path.join(artifact_dir, 'model_epochbest.pth')

# Load the state dictionary
state_dict = torch.load(model_path)

model = BartForSequenceClassification(5)

# Apply the state dictionary to the model
model.load_state_dict(state_dict)

In [1]:
# Now the model is ready to use
model.eval()  # Set the model to evaluation mode

predictions = []
with torch.no_grad():
    predictions = model(**tokenized).argmax(axis=1)

NameError: name 'model' is not defined

In [10]:
# 'class'를 'type'으로 매핑하는 딕셔너리 생성하기
class_to_type = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3,
    '일반 대화': 4
}
type_to_class = {key: idx for idx, key in class_to_type.items()}

In [11]:
class_predictions = [type_to_class[pred] for pred in predictions]

In [12]:
pred_df=pd.DataFrame(class_predictions, columns=['class'])
pred_df['file_name'] = submission_data.index
pred_df

,class,file_name
0,갈취 대화,t_000
1,갈취 대화,t_001
2,갈취 대화,t_002
3,갈취 대화,t_003
4,기타 괴롭힘 대화,t_004
...,...,...
495,직장 내 괴롭힘 대화,t_495
496,협박 대화,t_496
497,갈취 대화,t_497
498,갈취 대화,t_498


In [13]:
submission = pd.read_csv('./data/new_submission.csv')

In [14]:
submission = submission.drop('class', axis=1).merge(pred_df, on='file_name')
submission

,file_name,class
0,t_000,갈취 대화
1,t_001,갈취 대화
2,t_002,갈취 대화
3,t_003,갈취 대화
4,t_004,기타 괴롭힘 대화
...,...,...
495,t_495,직장 내 괴롭힘 대화
496,t_496,협박 대화
497,t_497,갈취 대화
498,t_498,갈취 대화


In [15]:
submission.to_csv('minions_submission.csv', index=False)